## Working with A5 in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/03_a5.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/03_a5.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/03_a5.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/03_a5.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/03_a5.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2a5

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2a5

lat = 10.775276
lon = 106.706797
res = 12
a5_hex = latlon2a5(lat, lon, res)
a5_hex

### A5 to Shapely Polygon

In [ ]:
from vgrid.conversion.dggs2geo.a52geo import a52geo
a5_geo = a52geo(a5_hex)
a5_geo

### A5 to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.a52geo import a52geojson
import json
a5_geojson = a52geojson(a5_hex)
a5_geojson

### Vector to A5

In [ ]:
from vgrid.conversion.vector2dggs.vector2a5 import vector2a5

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
vector_to_a5 = vector2a5(file_path, compact=False, resolution=16, 
                predicate = "intersects", output_format="gpd")
# Visualize vector_to_a5
vector_to_a5.plot(edgecolor="white")

### A5 Compact

In [ ]:
from vgrid.conversion.dggscompact.a5compact import a5compact

a5_compacted = a5compact(vector_to_a5, a5_hex="a5", output_format="gpd")
a5_compacted.plot(edgecolor="white")

### A5 Expand

In [ ]:
from vgrid.conversion.dggscompact.a5compact import a5expand

a5_expanded = a5expand(vector_to_a5, resolution=17, output_format="gpd")
a5_expanded.plot(edgecolor="white")

### A5 Binning

In [ ]:
from vgrid.binning.a5bin import a5bin
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "variety"
a5_bin = a5bin(file_path, resolution=15, stats=stats, 
                numeric_field="confidence",
                # category="category",
                output_format="gpd")

a5_bin.plot(
    column=stats,        # numeric column to base the colors on
    cmap='Spectral_r',        # color scheme (matplotlib colormap)
    legend=True,  
    linewidth=0.2          # boundary width (optional)
)

### Raster to A5

#### Download and open raster

In [ ]:
from vgrid.utils.io import download_file
import rasterio
from rasterio.plot import show

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)
src = rasterio.open(raster_file, 'r')
print(src.meta)
show(src)

#### Convert raster to A5

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2a5 import raster2a5

raster_to_a5 =  raster2a5(raster_file, resolution=23, output_format="gpd")
# Visualize raster_to_a5
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

a5_layer = folium.GeoJson(
    raster_to_a5,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 0.5,
    },
    popup=folium.GeoJsonPopup(
        fields=["a5", "resolution", "band_1", "band_2", "band_3", "cell_area"],
        aliases=["A5 Hex", "Resolution", "Band 1", "Band 2", "Band 3", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(a5_layer.get_bounds())

# Display the map
m

### A5 Generator

In [ ]:
from vgrid.generator.a5grid import a5grid

a5_grid = a5grid(resolution=2,output_format="gpd")
# a5_grid = a5grid(resolution=16,bbox=[106.699007, 10.762811, 106.717674, 10.778649],output_format="gpd")  
a5_grid.plot(edgecolor='white')

### A5 Inspect

In [ ]:
from vgrid.stats.a5stats import a5inspect
resolution = 5
a5_inspect = a5inspect(resolution)
a5_inspect.head()

### Distribution of A5 Area Distortions

In [ ]:
from vgrid.stats.a5stats import a5_norm_area
a5_norm_area(a5_inspect)

### Distribution of A5 IPQ Compactness

In [ ]:
from vgrid.stats.a5stats import a5_compactness
a5_compactness(a5_inspect)

### A5 Statistics

In [ ]:
from vgrid.stats.a5stats import a5stats
a5stats('km')